In [6]:
import numpy as np

from __future__ import print_function
import numpy as np
from ortools.sat.python import cp_model

import sys
import keras
from keras import Sequential
from keras import backend as bck
from keras.datasets import mnist

# Solver Class
This class contains a simple solver written in Or-Tools. Constraint programming used in here is very fast and (i hope) easy to understand.

Constructor contains data about sudoku - grid size, cell size and two or-tools objects:
- model
- solver
Both of those are used to find the solution. There is no need for more complicated solver than a basic CpSolver (however some intmix could be used, maybe in future, who knows?).

get_values() is a method reading grid given to class (read from image) and inserting to solution.
printer() is just a print-to-console class.
solve() is... solving. Contains proper constraints for sudoku.

In [7]:
# I am a simple solver.
class Solver:
    def __init__(self):
        self.model = cp_model.CpModel()
        self.solver = cp_model.CpSolver()
        self.grid_size = 9
        self.cell_size = 3

    def get_values(self, solution, original_grid):
        for row in range(self.grid_size):
            for column in range(self.grid_size):
                if original_grid[row, column] == 0:
                    solution[row, column] = self.model.NewIntVar(1, 9, '(%i,%i)' % (row, column))
                else:
                    solution[row, column] = self.model.NewConstant(int(original_grid[row, column]))
        return solution

    def printer(self, solution):
        for i in range(self.grid_size):
            if i % 3 == 0:
                print("\n")
            for j in range(self.grid_size):
                if j % 3 == 0 and j != 0:
                    print(" ", end = " ")
                print(self.solver.Value(solution[i,j]), end=" ")
            print()

    def solve(self, grid):
        solutions = {}

        # Get ints from grid
        solutions = self.get_values(solutions, grid)

        ##---CONSTRAINTS---
        # Horizontals
        for row in range(self.grid_size):
            line_horizontal = []
            for column in range(self.grid_size):
                line_horizontal.append(solutions[row, column])
            self.model.AddAllDifferent(line_horizontal)

        # Verticals
        for column in range(self.grid_size):
            line_vertical = []
            for row in range(self.grid_size):
                line_vertical.append(solutions[row, column])
            self.model.AddAllDifferent(line_vertical)

        # Squares
        for row_idx in range(0, self.grid_size, self.cell_size):
            for col_idx in range(0, self.grid_size, self.cell_size):
                self.model.AddAllDifferent([solutions[row_idx + row, column] for column in range(col_idx, (col_idx + self.cell_size)) 
                                            for row in range(self.cell_size)])

        # Solve model
        status = self.solver.Solve(self.model)
        self.printer(solutions)


## Training
The class below is handling MNIST dataset and image adjustments, not much more to say.

In [8]:
# I am a training reader.

class TrainingReader:
    def __init__(self):
        self.num_classes = 10
        self.size_rows = 28
        self.size_cols = 28
        self.input_shape = ()
        self.x_test = []
        self.x_train = []
        self.y_test = []
        self.y_train = []

    def adjust_images(self, x_train, x_test):
        # Channels adjustment
        if bck.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, self.size_rows, self.size_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, self.size_rows, self.size_cols)
            self.input_shape = (1, self.size_rows, self.size_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], self.size_rows, self.size_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], self.size_rows, self.size_cols, 1)
            self.input_shape = (self.size_rows, self.size_cols, 1)
        # Type adjustment
        x_train = x_train.astype('float32') / 255
        x_test = x_test.astype('float32') / 255
        return x_train, x_test

    def create_labels(self, y_original):
        y_new = keras.utils.to_categorical(y_original, self.num_classes)
        return y_new

    def read_mnist(self):
        try:
            (x_train, y_train), (x_test, y_test) = mnist.load_data()
            self.x_train, self.x_test = self.adjust_images(x_train, x_test)
            self.y_train = self.create_labels(y_train)
            self.y_test = self.create_labels(y_test)
            print("MNIST dataset was read correctly!")
        except:
            print("Unexpected error in TrainingReader: ", sys.exc_info()[0])
            raise
        

# Model class
This is still work in progress. Currently ts just a simple network, final version will probably contain much more - optimizer, training, model creating classes

In [9]:
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import Sequential
import sys

class Model:
    def create_model(self):
        try:
            model = Sequential()
            model.add(Conv2D(filters=32, kernel_size=(3, 3),activation='relu', input_shape=(28,28,1)))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.20))
            model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))
            model.add(Conv2D(128,(3, 3), activation='relu',padding='same'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.30))
            model.add(Conv2D(256,(3, 3), activation='relu',padding='same'))
            model.add(Dropout(0.40))
            model.add(Conv2D(512,(3, 3), activation='relu',padding='same'))
            model.add(Dropout(0.50))
            model.add(Flatten())
            model.add(Dense(256, activation='relu'))
            model.add(Dropout(0.35))
            model.add(Dense(128, activation='relu'))
            model.add(Dropout(0.25))
            model.add(Dense(10, activation='softmax'))
            model.summary()
        except:
            print("Error while creating the model: ", sys.exc_info()[0])

# TODO
1. Reading numbers from images
2. Reading lines from images (Hough lines)

# Very simple main
That's also work in progress.

In [10]:
if __name__ == "__main__":

    print("\n---------------")
    trainer = TrainingReader()
    trainer.read_mnist()

    print(trainer.input_shape)
    print("\n---------------")

    model_class = Model()
    model_class.create_model()

    grid =  np.array([[0,0,0, 2,6,0, 7,0,1],
                    [6,8,0, 0,7,0, 0,9,0],
                    [1,9,0, 0,0,4, 5,0,0],

                    [8,2,0, 1,0,0, 0,4,0],
                    [0,0,4, 6,0,2, 9,0,0],
                    [0,5,0, 0,0,3, 0,2,8],

                    [0,0,9, 3,0,0, 0,7,4],
                    [0,4,0, 0,5,0, 0,3,6],
                    [7,0,3, 0,1,8, 0,0,0]])


    print("\n---------------")
    sudoku_solver = Solver()
    print("Solving the sudoku...")
    sudoku_solver.solve(grid)
    print("\nHope you're happy with that solution!")
    print("\n---------------")



---------------
MNIST dataset was read correctly!
(28, 28, 1)

---------------
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
________________________________________